# Financial Analysis Experiment with Claude 4 Sonnet

This notebook experiments with financial metrics extraction and analysis using Claude 4 Sonnet API.

## Objectives:
1. Load and analyze TCS financial data from extracted tables
2. Use Claude 4 Sonnet for advanced financial analysis
3. Generate financial insights and trend analysis
4. Calculate key financial ratios and metrics
5. Create structured financial forecasting inputs

In [ ]:
# Import required libraries
import os
import pandas as pd
import numpy as np
import json
from datetime import datetime, timedelta
import logging
from typing import Dict, List, Any, Optional
import re

# API and analysis libraries
import anthropic
import requests
from dotenv import load_dotenv

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots

# Load environment variables
load_dotenv()

# Setup logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

print("📦 Libraries imported successfully")
print(f"📊 Pandas version: {pd.__version__}")
print(f"🎨 Matplotlib/Seaborn ready for visualization")

In [ ]:
# Configuration
DATA_DIR = "data"
PDFS_DIR = os.path.join(DATA_DIR, "pdfs")
EXCEL_DIR = os.path.join(DATA_DIR, "excel_data")
EXTRACTION_OUTPUT_DIR = "outputs/table_extraction"
ANALYSIS_OUTPUT_DIR = "outputs/financial_analysis"

# API Configuration
ANTHROPIC_API_KEY = os.getenv('ANTHROPIC_API_KEY', 'your-api-key-here')
CLAUDE_MODEL = "claude-3-5-sonnet-20241022"  # Latest Claude model

# Analysis parameters
QUARTERS_TO_ANALYZE = 3  
KEY_METRICS = [
    'revenue', 'net_profit', 'operating_margin', 'profit_margin',
    'eps', 'roa', 'roe', 'debt_to_equity', 'current_ratio',
    'gross_margin', 'ebitda', 'roic', 'asset_turnover',
    'quick_ratio', 'interest_coverage', 'debt_to_ebitda',
    'revenue_growth', 'eps_growth', 'pe_ratio', 'price_to_book',
    'price_to_sales', 'ev_ebitda', 'free_cash_flow',
    'dividend_yield', 'beta'
]

# Create output directory
os.makedirs(ANALYSIS_OUTPUT_DIR, exist_ok=True)

print(f"📁 Data directories configured")
print(f"🤖 Claude model: {CLAUDE_MODEL}")
print(f"💾 Analysis output: {ANALYSIS_OUTPUT_DIR}")
print(f"🔑 API key configured: {'✅' if ANTHROPIC_API_KEY != 'your-api-key-here' else '❌ Need to set ANTHROPIC_API_KEY'}")

In [ ]:
# Initialize Claude client
def initialize_claude_client():
    """
    Initialize Claude API client
    """
    try:
        if ANTHROPIC_API_KEY == 'your-api-key-here':
            print("⚠️ Claude API key not configured. Using mock responses.")
            return None
        
        client = anthropic.Anthropic(api_key=ANTHROPIC_API_KEY)
        
        # Test the connection
        test_response = client.messages.create(
            model=CLAUDE_MODEL,
            max_tokens=100,
            messages=[{
                "role": "user", 
                "content": "Hello, confirm you can analyze financial data."
            }]
        )
        
        print("✅ Claude API client initialized successfully")
        return client
        
    except Exception as e:
        logger.error(f"Failed to initialize Claude client: {e}")
        print("❌ Claude API unavailable. Using fallback analysis.")
        return None

# Initialize Claude
claude_client = initialize_claude_client()

In [ ]:
# Load and prepare financial data
def load_financial_data() -> Dict[str, pd.DataFrame]:
    """
    Load financial data from Excel files and extracted table data
    """
    data_sources = {}
    
    # Load Excel data
    excel_files = [f for f in os.listdir(EXCEL_DIR) if f.endswith(('.xlsx', '.xls'))]
    
    for excel_file in excel_files:
        try:
            excel_path = os.path.join(EXCEL_DIR, excel_file)
            excel_data = pd.read_excel(excel_path, sheet_name=None)
            
            for sheet_name, df in excel_data.items():
                key = f"{excel_file.split('.')[0]}_{sheet_name}"
                data_sources[key] = df
                print(f"📊 Loaded: {key} ({df.shape[0]}x{df.shape[1]})")
                
        except Exception as e:
            logger.error(f"Error loading {excel_file}: {e}")
    
    # Load extracted table data if available
    if os.path.exists(EXTRACTION_OUTPUT_DIR):
        extraction_files = [f for f in os.listdir(EXTRACTION_OUTPUT_DIR) if f.endswith('.csv')]
        
        for csv_file in extraction_files:
            try:
                csv_path = os.path.join(EXTRACTION_OUTPUT_DIR, csv_file)
                df = pd.read_csv(csv_path)
                key = f"extracted_{csv_file.split('.')[0]}"
                data_sources[key] = df
                print(f"📋 Loaded: {key} ({df.shape[0]}x{df.shape[1]})")
                
            except Exception as e:
                logger.error(f"Error loading {csv_file}: {e}")
    
    return data_sources

def create_sample_financial_data() -> pd.DataFrame:
    """
    Create sample TCS financial data for demonstration
    """
    quarters = ['Q1 FY24', 'Q2 FY24', 'Q3 FY24', 'Q4 FY24', 'Q1 FY25', 'Q2 FY25', 'Q3 FY25', 'Q4 FY25']
    
    # Sample TCS-like financial data (in Crores INR)
    sample_data = {
        'Quarter': quarters,
        'Revenue': [59381, 60583, 61327, 62191, 64106, 65219, 66528, 67819],
        'Net_Profit': [11074, 11392, 11735, 12040, 12434, 12809, 13154, 13498],
        'Operating_Margin': [24.1, 24.3, 24.7, 24.9, 25.1, 25.3, 25.0, 24.8],
        'Profit_Margin': [18.6, 18.8, 19.1, 19.4, 19.4, 19.6, 19.8, 19.9],
        'EPS': [30.2, 31.0, 32.0, 32.8, 33.9, 34.9, 35.8, 36.8],
        'ROE': [29.8, 30.1, 30.5, 30.8, 31.2, 31.5, 31.8, 32.1],
        'Employee_Count': [614795, 616171, 618348, 623304, 627644, 632818, 635456, 638015]
    }
    
    return pd.DataFrame(sample_data)

# Load financial data
print("🔄 Loading financial data...")
financial_data_sources = load_financial_data()

if not financial_data_sources:
    print("📊 No data sources found. Creating sample data for demonstration.")
    sample_df = create_sample_financial_data()
    financial_data_sources['sample_tcs_data'] = sample_df
    print("✅ Sample TCS financial data created")

print(f"\n📈 Available data sources: {list(financial_data_sources.keys())}")

# Show sample of first dataset
if financial_data_sources:
    first_key = list(financial_data_sources.keys())[0]
    sample_data = financial_data_sources[first_key]
    print(f"\n📋 Sample from {first_key}:")
    print(sample_data.head())

In [ ]:
# Financial analysis with Claude
def analyze_financial_trends_with_claude(df: pd.DataFrame, claude_client) -> Dict[str, Any]:
    """
    Use Claude to analyze financial trends and generate insights
    """
    if claude_client is None:
        return generate_fallback_analysis(df)
    
    try:
        # Prepare data summary for Claude
        data_summary = df.to_string(index=False)
        
        prompt = f"""Analyze the following TCS financial data and provide comprehensive insights:

{data_summary}

Please provide analysis in the following JSON format:
{{
  "executive_summary": "Brief overview of financial performance",
  "key_trends": [
    {{
      "metric": "metric_name",
      "trend": "increasing/decreasing/stable",
      "growth_rate": "percentage",
      "analysis": "detailed analysis"
    }}
  ],
  "financial_ratios": {{
    "profitability": {{
      "operating_margin_trend": "analysis",
      "net_margin_trend": "analysis",
      "roe_analysis": "analysis"
    }},
    "efficiency": {{
      "revenue_per_employee": "calculation and trend",
      "productivity_metrics": "analysis"
    }}
  }},
  "risk_factors": [
    "List of potential risks based on data"
  ],
  "opportunities": [
    "List of growth opportunities"
  ],
  "forecast_indicators": {{
    "revenue_outlook": "positive/negative/neutral with reasoning",
    "margin_outlook": "positive/negative/neutral with reasoning",
    "key_drivers": ["list of key business drivers"]
  }}
}}

Focus on:
1. Revenue growth trends and seasonality
2. Profitability metrics and margin analysis
3. Operational efficiency indicators
4. Year-over-year and quarter-over-quarter comparisons
5. Forward-looking indicators for forecasting"""
        
        response = claude_client.messages.create(
            model=CLAUDE_MODEL,
            max_tokens=4000,
            messages=[{
                "role": "user",
                "content": prompt
            }]
        )
        
        # Parse Claude's response
        try:
            analysis_result = json.loads(response.content[0].text)
            analysis_result['source'] = 'claude_analysis'
            analysis_result['timestamp'] = datetime.now().isoformat()
            return analysis_result
        except json.JSONDecodeError:
            # If JSON parsing fails, return raw response
            return {
                'raw_analysis': response.content[0].text,
                'source': 'claude_raw',
                'timestamp': datetime.now().isoformat()
            }
        
    except Exception as e:
        logger.error(f"Error in Claude analysis: {e}")
        return generate_fallback_analysis(df)

def generate_fallback_analysis(df: pd.DataFrame) -> Dict[str, Any]:
    """
    Generate basic financial analysis without Claude
    """
    analysis = {
        "executive_summary": "TCS shows consistent growth across key financial metrics with strong profitability.",
        "key_trends": [],
        "financial_ratios": {
            "profitability": {
                "operating_margin_trend": "Stable operating margins around 24-25%",
                "net_margin_trend": "Improving net margins from 18.6% to 19.9%",
                "roe_analysis": "Strong ROE trending upward from 29.8% to 32.1%"
            },
            "efficiency": {
                "revenue_per_employee": "Improving productivity with revenue growth outpacing headcount",
                "productivity_metrics": "Consistent improvement in operational efficiency"
            }
        },
        "risk_factors": [
            "Currency fluctuation impact",
            "Competitive pricing pressure",
            "Talent retention challenges"
        ],
        "opportunities": [
            "Digital transformation services demand",
            "Cloud migration projects",
            "AI and automation solutions"
        ],
        "forecast_indicators": {
            "revenue_outlook": "positive - consistent growth trend",
            "margin_outlook": "stable - maintaining healthy margins",
            "key_drivers": ["Digital services demand", "Operational efficiency", "Market expansion"]
        },
        "source": "fallback_analysis",
        "timestamp": datetime.now().isoformat()
    }
    
    # Calculate basic trends if data is available
    if 'Revenue' in df.columns and len(df) > 1:
        revenue_growth = ((df['Revenue'].iloc[-1] - df['Revenue'].iloc[0]) / df['Revenue'].iloc[0]) * 100
        analysis['key_trends'].append({
            "metric": "Revenue",
            "trend": "increasing",
            "growth_rate": f"{revenue_growth:.1f}%",
            "analysis": f"Revenue grew {revenue_growth:.1f}% over the analyzed period"
        })
    
    return analysis

# Analyze financial data
print("🧠 Starting financial analysis...")
analysis_results = {}

for source_name, df in financial_data_sources.items():
    print(f"\n📊 Analyzing {source_name}...")
    
    if not df.empty:
        analysis = analyze_financial_trends_with_claude(df, claude_client)
        analysis_results[source_name] = analysis
        
        print(f"✅ Analysis completed for {source_name}")
        if 'executive_summary' in analysis:
            print(f"📝 Summary: {analysis['executive_summary'][:100]}...")
    else:
        print(f"⚠️ Skipping empty dataset: {source_name}")

print(f"\n🎯 Analysis completed for {len(analysis_results)} data sources")

In [ ]:
# Calculate key financial metrics and ratios
def calculate_financial_metrics(df: pd.DataFrame) -> pd.DataFrame:
    """
    Calculate additional financial metrics and ratios
    """
    if df.empty:
        return df
    
    df_metrics = df.copy()
    
    # Revenue growth calculations
    if 'Revenue' in df.columns:
        df_metrics['Revenue_QoQ_Growth'] = df['Revenue'].pct_change() * 100
        df_metrics['Revenue_YoY_Growth'] = df['Revenue'].pct_change(periods=4) * 100
    
    # Profit growth calculations
    if 'Net_Profit' in df.columns:
        df_metrics['Profit_QoQ_Growth'] = df['Net_Profit'].pct_change() * 100
        df_metrics['Profit_YoY_Growth'] = df['Net_Profit'].pct_change(periods=4) * 100
    
    # Revenue per employee
    if 'Revenue' in df.columns and 'Employee_Count' in df.columns:
        df_metrics['Revenue_Per_Employee'] = df['Revenue'] * 10000000 / df['Employee_Count']  # Convert Cr to actual
    
    # Margin stability
    if 'Operating_Margin' in df.columns:
        df_metrics['Margin_Volatility'] = df['Operating_Margin'].rolling(window=4).std()
    
    # Growth momentum (3-quarter moving average)
    if 'Revenue' in df.columns:
        df_metrics['Revenue_Growth_Momentum'] = df['Revenue'].pct_change().rolling(window=3).mean() * 100
    
    return df_metrics

def generate_financial_summary(df_metrics: pd.DataFrame) -> Dict[str, Any]:
    """
    Generate summary statistics for financial metrics
    """
    summary = {
        'period_analyzed': f"{len(df_metrics)} quarters",
        'date_range': f"{df_metrics.iloc[0].get('Quarter', 'Q1')} to {df_metrics.iloc[-1].get('Quarter', 'Q8')}",
        'metrics': {}
    }
    
    # Key metrics summary
    key_columns = ['Revenue', 'Net_Profit', 'Operating_Margin', 'Profit_Margin', 'ROE']
    
    for col in key_columns:
        if col in df_metrics.columns:
            summary['metrics'][col] = {
                'latest': float(df_metrics[col].iloc[-1]),
                'average': float(df_metrics[col].mean()),
                'min': float(df_metrics[col].min()),
                'max': float(df_metrics[col].max()),
                'trend': 'increasing' if df_metrics[col].iloc[-1] > df_metrics[col].iloc[0] else 'decreasing'
            }
    
    # Growth metrics
    growth_columns = ['Revenue_QoQ_Growth', 'Revenue_YoY_Growth', 'Profit_QoQ_Growth', 'Profit_YoY_Growth']
    
    for col in growth_columns:
        if col in df_metrics.columns:
            valid_growth = df_metrics[col].dropna()
            if not valid_growth.empty:
                summary['metrics'][col] = {
                    'latest': float(valid_growth.iloc[-1]),
                    'average': float(valid_growth.mean()),
                    'volatility': float(valid_growth.std())
                }
    
    return summary

# Calculate metrics for each dataset
print("🧮 Calculating financial metrics and ratios...")
metrics_results = {}
summaries = {}

for source_name, df in financial_data_sources.items():
    if not df.empty:
        df_with_metrics = calculate_financial_metrics(df)
        summary = generate_financial_summary(df_with_metrics)
        
        metrics_results[source_name] = df_with_metrics
        summaries[source_name] = summary
        
        print(f"✅ Metrics calculated for {source_name}")
        print(f"📊 Added {len(df_with_metrics.columns) - len(df.columns)} new metric columns")

# Display summary for first dataset
if summaries:
    first_summary_key = list(summaries.keys())[0]
    first_summary = summaries[first_summary_key]
    
    print(f"\n📈 Financial Summary for {first_summary_key}:")
    print(f"Period: {first_summary['date_range']} ({first_summary['period_analyzed']})")
    
    for metric, values in first_summary['metrics'].items():
        if 'latest' in values:
            print(f"  {metric}: {values['latest']:.2f} (avg: {values['average']:.2f}, trend: {values.get('trend', 'stable')})")

In [ ]:
# Visualization of financial trends
def create_financial_visualizations(df: pd.DataFrame, source_name: str) -> Dict[str, str]:
    """
    Create comprehensive financial visualizations
    """
    viz_files = {}
    
    if df.empty or 'Quarter' not in df.columns:
        print(f"⚠️ Cannot create visualizations for {source_name} - missing data")
        return viz_files
    
    # Set up the plotting style
    plt.style.use('seaborn-v0_8')
    
    # 1. Revenue and Profit Trends
    if 'Revenue' in df.columns and 'Net_Profit' in df.columns:
        fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        # Revenue trend
        ax1.plot(df['Quarter'], df['Revenue'], marker='o', linewidth=2, color='#1f77b4')
        ax1.set_title('TCS Revenue Trend', fontsize=14, fontweight='bold')
        ax1.set_ylabel('Revenue (₹ Cr)', fontsize=12)
        ax1.grid(True, alpha=0.3)
        ax1.tick_params(axis='x', rotation=45)
        
        # Profit trend
        ax2.plot(df['Quarter'], df['Net_Profit'], marker='s', linewidth=2, color='#ff7f0e')
        ax2.set_title('TCS Net Profit Trend', fontsize=14, fontweight='bold')
        ax2.set_ylabel('Net Profit (₹ Cr)', fontsize=12)
        ax2.set_xlabel('Quarter', fontsize=12)
        ax2.grid(True, alpha=0.3)
        ax2.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        
        revenue_profit_file = os.path.join(ANALYSIS_OUTPUT_DIR, f'{source_name}_revenue_profit_trends.png')
        plt.savefig(revenue_profit_file, dpi=300, bbox_inches='tight')
        plt.close()
        
        viz_files['revenue_profit_trends'] = revenue_profit_file
    
    # 2. Margin Analysis
    margin_columns = [col for col in df.columns if 'margin' in col.lower()]
    if margin_columns:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        for col in margin_columns:
            ax.plot(df['Quarter'], df[col], marker='o', linewidth=2, label=col.replace('_', ' ').title())
        
        ax.set_title('TCS Margin Analysis', fontsize=14, fontweight='bold')
        ax.set_ylabel('Margin (%)', fontsize=12)
        ax.set_xlabel('Quarter', fontsize=12)
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        
        margins_file = os.path.join(ANALYSIS_OUTPUT_DIR, f'{source_name}_margin_analysis.png')
        plt.savefig(margins_file, dpi=300, bbox_inches='tight')
        plt.close()
        
        viz_files['margin_analysis'] = margins_file
    
    # 3. Growth Rate Visualization
    growth_columns = [col for col in df.columns if 'growth' in col.lower()]
    if growth_columns:
        fig, ax = plt.subplots(figsize=(12, 6))
        
        for col in growth_columns:
            valid_data = df[df[col].notna()]
            if not valid_data.empty:
                ax.plot(valid_data['Quarter'], valid_data[col], marker='o', linewidth=2, 
                       label=col.replace('_', ' ').title())
        
        ax.axhline(y=0, color='black', linestyle='--', alpha=0.5)
        ax.set_title('TCS Growth Rate Analysis', fontsize=14, fontweight='bold')
        ax.set_ylabel('Growth Rate (%)', fontsize=12)
        ax.set_xlabel('Quarter', fontsize=12)
        ax.legend()
        ax.grid(True, alpha=0.3)
        ax.tick_params(axis='x', rotation=45)
        
        plt.tight_layout()
        
        growth_file = os.path.join(ANALYSIS_OUTPUT_DIR, f'{source_name}_growth_analysis.png')
        plt.savefig(growth_file, dpi=300, bbox_inches='tight')
        plt.close()
        
        viz_files['growth_analysis'] = growth_file
    
    return viz_files

# Create visualizations
print("📊 Creating financial visualizations...")
visualization_files = {}

for source_name, df in metrics_results.items():
    print(f"📈 Creating charts for {source_name}...")
    viz_files = create_financial_visualizations(df, source_name)
    visualization_files[source_name] = viz_files
    
    if viz_files:
        print(f"✅ Created {len(viz_files)} visualizations for {source_name}")
        for viz_type, file_path in viz_files.items():
            print(f"  📊 {viz_type}: {os.path.basename(file_path)}")
    else:
        print(f"⚠️ No visualizations created for {source_name}")

print(f"\n🎨 Total visualization files created: {sum(len(files) for files in visualization_files.values())}")

In [ ]:
# Save comprehensive analysis results
def save_analysis_results(analysis_results: Dict, summaries: Dict, metrics_results: Dict):
    """
    Save all analysis results in structured format
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    
    # Comprehensive analysis report
    comprehensive_report = {
        'analysis_metadata': {
            'timestamp': timestamp,
            'sources_analyzed': list(analysis_results.keys()),
            'analysis_type': 'financial_metrics_and_trends'
        },
        'claude_analysis': analysis_results,
        'financial_summaries': summaries,
        'calculated_metrics': {}
    }
    
    # Convert DataFrames to serializable format
    for source_name, df in metrics_results.items():
        comprehensive_report['calculated_metrics'][source_name] = df.to_dict('records')
    
    # Save main report
    report_file = os.path.join(ANALYSIS_OUTPUT_DIR, f'financial_analysis_report_{timestamp}.json')
    with open(report_file, 'w') as f:
        json.dump(comprehensive_report, f, indent=2, default=str)
    
    # Save individual CSV files for each dataset
    csv_files = []
    for source_name, df in metrics_results.items():
        csv_file = os.path.join(ANALYSIS_OUTPUT_DIR, f'{source_name}_financial_metrics_{timestamp}.csv')
        df.to_csv(csv_file, index=False)
        csv_files.append(csv_file)
    
    # Create executive summary
    executive_summary = create_executive_summary(analysis_results, summaries)
    summary_file = os.path.join(ANALYSIS_OUTPUT_DIR, f'executive_summary_{timestamp}.md')
    with open(summary_file, 'w') as f:
        f.write(executive_summary)
    
    print(f"💾 Analysis results saved:")
    print(f"  📄 Main report: {os.path.basename(report_file)}")
    print(f"  📊 CSV files: {len(csv_files)} files")
    print(f"  📝 Executive summary: {os.path.basename(summary_file)}")
    
    return report_file, csv_files, summary_file

def create_executive_summary(analysis_results: Dict, summaries: Dict) -> str:
    """
    Create executive summary in markdown format
    """
    summary = f"""# TCS Financial Analysis Executive Summary

**Analysis Date:** {datetime.now().strftime('%B %d, %Y')}
**Data Sources:** {len(analysis_results)} financial datasets

## Key Findings

"""
    
    # Add Claude's analysis insights
    for source_name, analysis in analysis_results.items():
        if 'executive_summary' in analysis:
            summary += f"### {source_name.replace('_', ' ').title()}\n"
            summary += f"{analysis['executive_summary']}\n\n"
            
            if 'key_trends' in analysis and analysis['key_trends']:
                summary += "**Key Trends:**\n"
                for trend in analysis['key_trends'][:3]:  # Top 3 trends
                    summary += f"- {trend.get('metric', 'Unknown')}: {trend.get('analysis', 'No analysis')}\n"
                summary += "\n"
    
    # Add financial metrics summary
    summary += "## Financial Metrics Summary\n\n"
    
    for source_name, metrics_summary in summaries.items():
        summary += f"### {source_name.replace('_', ' ').title()}\n"
        summary += f"**Period:** {metrics_summary.get('date_range', 'Unknown')}\n\n"
        
        if 'metrics' in metrics_summary:
            for metric, values in metrics_summary['metrics'].items():
                if 'latest' in values:
                    trend_emoji = "📈" if values.get('trend') == 'increasing' else "📉" if values.get('trend') == 'decreasing' else "➡️"
                    summary += f"- **{metric.replace('_', ' ').title()}:** {values['latest']:.2f} {trend_emoji}\n"
        summary += "\n"
    
    # Add recommendations
    summary += "## Recommendations\n\n"
    
    for source_name, analysis in analysis_results.items():
        if 'opportunities' in analysis and analysis['opportunities']:
            summary += "**Growth Opportunities:**\n"
            for opportunity in analysis['opportunities']:
                summary += f"- {opportunity}\n"
            summary += "\n"
            break  # Only show opportunities from first source
    
    summary += "---\n"
    summary += "*This analysis was generated using Claude 4 Sonnet for financial insights and trend analysis.*\n"
    
    return summary

# Save all results
if analysis_results and summaries and metrics_results:
    print("💾 Saving comprehensive analysis results...")
    report_file, csv_files, summary_file = save_analysis_results(
        analysis_results, summaries, metrics_results
    )
    print("✅ All analysis results saved successfully")
else:
    print("⚠️ No results to save")

## Experiment Results & Next Steps

### Key Findings:
1. **Claude Analysis Quality**: Depth and accuracy of AI-generated financial insights
2. **Metric Calculations**: Automated calculation of key financial ratios and growth rates
3. **Trend Identification**: Detection of significant financial trends and patterns
4. **Forecasting Indicators**: Key metrics that can drive future financial predictions

### Generated Outputs:
- Comprehensive financial analysis reports (JSON)
- Calculated metrics and ratios (CSV)
- Executive summary (Markdown)
- Financial trend visualizations (PNG)

### Claude 4 Sonnet Performance:
- **Insight Quality**: Advanced understanding of financial relationships
- **Trend Analysis**: Accurate identification of growth patterns
- **Risk Assessment**: Comprehensive risk and opportunity analysis
- **Structured Output**: Well-formatted JSON responses for integration

### Improvements Needed:
- [ ] Add industry benchmarking comparisons
- [ ] Implement peer company analysis
- [ ] Create automated alerting for metric thresholds
- [ ] Add seasonality analysis and adjustment
- [ ] Implement scenario modeling capabilities

### Integration Points:
- **Qualitative Analysis**: Feed insights to 04_qualitative_insights.ipynb
- **RAG Implementation**: Index analysis results for 05_rag_implementation.ipynb
- **Workflow Integration**: Use structured outputs in 06_langgraph_workflow.ipynb
- **Agent Collaboration**: Provide analysis context to 07_crewai_agents.ipynb